In [6]:
# Import libraries and needed classes

import numpy as np
import sys
import os

from Code.RNN.layers.sigmoid import SigmoidActivation
from Code.RNN.functions.metrics import mse, accuracy
from Code.RNN.networks.recorrent_neural_network import RecorrentNeuralNetwork
from Code.RNN.functions.mse import MeanSquaredError
from Code.RNN.layers.rnn import RNN
from Code.RNN.layers.dense import DenseLayer
from Code.RNN.optimizations.retained_gradient import RetGradient
from Code.DNN.optimizations.l2_reg import L2Reg
from Code.RNN.functions.bce import BinaryCrossEntropy
from Code.RNN.layers.relu import ReLUActivation

from Code.utils.dataset import Dataset
import Code.utils.store_model as store_model

In [7]:
# Set random seed to control randomness

np.random.seed(42)

In [8]:
# Read datasets

dataset = Dataset('../../Dataset/DatasetsGerados/dataset_training_input.csv',
                      '../../Dataset/DatasetsGerados/dataset_training_output.csv',
                      '../../Dataset/DatasetsGerados/dataset_validation_input.csv',
                      '../../Dataset/DatasetsGerados/dataset_validation_output.csv',
                      '../../Dataset/DatasetsGerados/dataset_test_input.csv',
                      '../../Dataset/DatasetsGerados/dataset_test_output.csv')

X_train, y_train, X_validation, y_validation, X_test, y_test, ids = dataset.get_datasets('Text', 'Label', sep='\t', rem_punctuation=False)

In [9]:
# Build network

timestep = 2
batch_size = 8

# network
optimizer = RetGradient(learning_rate=0.005, momentum=0.90)
loss = BinaryCrossEntropy()

regulator = L2Reg(l2_val=0.001)
net = RecorrentNeuralNetwork(epochs=3, batch_size=batch_size, optimizer=optimizer, regulator=regulator, verbose=True, loss=loss,
                    metric=accuracy, patience=2, min_delta=0.001, timestep=timestep)

net.add(RNN(5, input_shape=(timestep, X_train.shape[1])))
net.add(SigmoidActivation())
net.add(DenseLayer(1, timestep))
net.add(SigmoidActivation())

In [10]:
# Train network

net.fit(X_train, y_train, X_val=X_validation, y_val=y_validation)

# Plot learning curves
net.plot_train_curves()

AssertionError: X's number of rows should be divisible by batch_size

In [12]:
# Test network

out = net.predict(X_test)
out = out.reshape(out.shape[1], 1)

if y_test is not None:
    print(net.score(y_test, out))

0.95


In [7]:
# Store results

store_results = './Results/rnn_results.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(store_results), exist_ok=True)

results = dataset.merge_results(ids, out)
results.to_csv(store_results, sep='\t', index=False)

In [8]:
# Store network model

model_filepath = './Model/rnn'
model_key = 'rnn'

store_model.store_model(model_filepath, model_key, net)

In [9]:
# Retrieve network model

model_filepath = './Model/rnn'
model_key = 'rnn'

net = store_model.retrieve_model(model_filepath, model_key)